In [131]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

In [132]:
driver = webdriver.Chrome('C:\chromedriver\chromedriver')

In [133]:
def get_data(item):
    
    #grab the title
    title = item.h2.a.text.strip()
    
    #grab the ratings and review data
    try:
        rating = item.i.span.text.strip()
        num_review = item.find('span', attrs = {"class":"a-size-base"}).text
    except AttributeError:
        return
    
    #grab the price data
    try: 
        price_parent = item.find('span','a-price')
        price = price_parent.find('span','a-offscreen').text
        price = float(price.replace("£","").replace(",",""))
    except AttributeError:
        return 
    
    #grab the product specifcation data
    try:
        parent_specs = item.find('div','s-product-specs-view')
        specs = parent_specs.find_all('span','a-text-bold')
        display_size = specs[0].text
        storage_type = specs[1].text
        cpu_speed = specs[2].text
        ram = specs[3].text
    except AttributeError:
        return
    
    if(price>100):
        result = (title, price, num_review, display_size, storage_type, cpu_speed, ram,rating)
        return result
    else:
        return 

In [134]:
records = []

for i in range(1,400):
    
    #url of amazon search results for laptops
    url = "https://www.amazon.co.uk/s?k=laptop&i=computers&page={}&crid=2HCWD0O77NCOW&qid=1617022693&sprefix=laptop%2Caps%2C174&ref=sr_pg_2".format(i)
    driver.get(url)
    
    #beautiful soup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    results = soup.find_all('div', {'data-component-type' : 's-search-result'})
    
    #extracting the needed data
    for item in results:
        data = get_data(item)
        if (data):
            records.append(data)


In [135]:
with open('laptop_data.csv','w', newline ="",encoding = 'utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Title', 'Price', 'No. Reviews', 'Display size', 'Storage type', 'CPU Speed', 'RAM', 'Rating'])
    writer.writerows(records)

In [136]:
driver.close()